In [1]:
%%html
<style>table {float:left}</style>

In [2]:
import sys
print('python version:', sys.version_info)
import sqlalchemy
print('sqlalchemy version:',sqlalchemy.__version__)
import sqlalchemy.orm

python version: sys.version_info(major=3, minor=9, micro=18, releaselevel='final', serial=0)
sqlalchemy version: 2.0.21


 # Pythonとデータベース
Pythonから利用できるデータベースはMySQL,PostgreSQL, SQLite,Oracleなど多岐にわたります。
PythonのデータベースAPI仕様は、PEP 249※1で定義されており、それぞれのデータベースに対応して実装※2が存在します。

データベースAPIを使用すればデータベースプログラミンが可能ですが、SQLを直接記述する必要があります。
SQLを直接記述するプログラムは規模が大きくなると複雑になる傾向があります。そのため最近のデータベース開発では
オブジェクトとデータベースのレコードを関連付けるしくみであるO/Rマッパーを利用してデータベース開発を行うケースが増えています。

※1 https://www.python.org/dev/peps/pep-0249/#exceptions <br>
※2 https://docs.python.org/ja/3/library/sqlite3.html

# SQLAlchemy
SQLAlchemyは、データベースやSQLに関連する機能を提供するライブラリです。テーブル関連の処理やSQLを扱うための便利なモジュールを提供しています。
O/Rマッパーは提供される機能の１つであり、O/Rマッパーを使用せずに開発することも可能です。SQLAlchemyではDataMapperパターンを使用してテーブルとオブジェクトのマッピングを行い、開発を進めます。

SQLAlchmeyは非常に高機能であり、SQLite,MySQL,PostgerSQLなどのオープンソースのデータベースや、OracleやMicrosoft SQL Serverなどの商用データベースもサポートしています。

# データベースへの接続
データベースの接続は、create_engine()関数を使用します。第１引数は接続するデータベースを指定します。ここでは、メモリーベースのSQLite3を指定しています。
第2引数のechoはログ出力の有効化フラグです。Trueを指定するとコンソールにSQLAlchemyが発行したSQLなどが出力されます。

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
engine

Engine(sqlite:///:memory:)

# テーブルの作成
SQLAlchemyを使用してテーブルを作成します。テーブルは次の表のように書籍の情報を格納するbooksテーブルを作成します。SQLAlchemyでは、テーブルを表すオブジェクトとマッピングされるオブジェクトを定義し、それらを関連付けます。
定義と関連付けはdeclarativeを使って同時に行っています。
#### booksテーブルの定義
|<div align='left'>名前</div>|<div align='left'>カラム名</div>|<div align='left'>データタイプ</div>
|-----|-----|-----
|<div align='left'>ID</div>|<div align='left'>id</div>|<div align='left'>INTEGER</div>
|<div align='left'>タイトル</div>|<div align='left'>title</div>|<div align='left'>VARCHAR(255)</div>
|<div align='left'>価格</div>|<div align='left'>price</div>|<div align='left'>INTEGER</div>
|<div align='left'>メモ</div>|<div align='left'>memo</div>|<div align='left'>TEXT</div>

### Bookクラス（オブジェクト）定義

Bookクラスの定義は次のように行います。
1. declarativeをインポートします。
2. declarative.declarative_base()関数でBaseクラスを生成します。
3. フィールド定義に使用する Column, Integer, Unicode, UnicodeTextをインポートします。
4. Baseクラスを継承したBookクラスを生成します。Bookクラスでは、データベースのカラムに対応するフィールドの定義を行います。
5. \_\_tablename\_\_ = 'books' でテーブル名を指定します。
6. Columnクラスを使ってフィールドを定義します。

Columnクラスのコンストラクタの第１引数には、フィールドの型、第２引数以降にはオプションを指定します。
primary_key=Trueと指定すると、そのカラムはプライマリーキーとなります。nullable=Falseと指定すると、nullが許可さなくなります。
また、テーブルにデータが保存されるときのでフィルと値はdefault='<デフォルト値>'で指定します。

declarativeを使ってクラス宣言したので、booksテーブルとマッピングされたBookオブジェクトがプログラム内で使用できるようになります。


In [9]:
# from sqlalchemy.ext import declarative
# Base = declarative.declarative_base()
import sqlalchemy.orm
Base = sqlalchemy.orm.declarative_base()
from sqlalchemy import Column, Integer, Unicode, UnicodeText
class Book(Base):
    __tablename__ = 'books'
    id = Column(Integer, primary_key=True)
    title = Column(Unicode(100), nullable=False)
    price = Column(Integer, nullable=False)
    memo = Column(UnicodeText)
    def __repr__(self):
        return "<Book('%s', '%s', '%s')>" % (self.title, self.price, self.memo)

### テーブル作成
テーブルは、Base.metadatのcreate_all()メソッドにcreate_engine() で作成したengineを渡して作成します。
Metadateはテーブルなどの情報を登録するためのオブジェクトです。create_all()メソッドを呼び出すとデータベース上にテーブルが作成されます。

In [10]:
Base.metadata.create_all(engine)

2024-02-09 14:28:23,522 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-09 14:28:23,525 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("books")
2024-02-09 14:28:23,527 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-09 14:28:23,529 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("books")
2024-02-09 14:28:23,529 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-09 14:28:23,529 INFO sqlalchemy.engine.Engine 
CREATE TABLE books (
	id INTEGER NOT NULL, 
	title VARCHAR(100) NOT NULL, 
	price INTEGER NOT NULL, 
	memo TEXT, 
	PRIMARY KEY (id)
)


2024-02-09 14:28:23,529 INFO sqlalchemy.engine.Engine [no key 0.00160s] ()
2024-02-09 14:28:23,541 INFO sqlalchemy.engine.Engine COMMIT


# テーブルの操作
テーブルとオブジェクトのマッピングが完了すると、データベースにアクセスして基本的なCRUD操作ができるようになります。<br>
CRUDとは、**C**reate(登録)、 **R**ead（読み取り）、**U**pdate(更新）、**D**elete（削除）の略称です。
### セッションの作成
SQLAlchemyではセッションを通してオブジェクトの取得や更新を行います。まずは次に示すようにSessionインスタンスを作成します。
1. sqlchemey.ormのsessionmakerをインポートします。
2. sessionmakerでengineを関連付け(bind)してSessionクラスを作成します。
3. Sessionクラスのコンストラクターを呼び出してインスタンスsessionを生成します。

In [4]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

### トランザクション
SQLAlchemyでは、オブジェクトの登録、更新、削除を行った後、session.commit()メソッド呼び出してオブジェクトの更新をデータベースに反映します。
また何らかのエラーが発生してトランザクションをロールバックしたい場合には、session.rollback()メソッドを呼び出します。

### C. オブジェクトの登録
セッションを生成したら、Bookクラスのオブジェクトをデータベースに登録します。
オブジェクトを登録するにはBookクラスのインスタンスを生成し、**session\.add**()メソッドの第１引数に渡します。
Integer型のカラムにprimary_key=Trueが指定されていると、そのカラムは自動的に番号が付けられます。

In [12]:
book = Book(title= 'やさしいPython', price='2580', memo = 'SBクリエイティブ')
session.add(book)
session.commit()

2024-02-09 14:28:48,914 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-09 14:28:48,920 INFO sqlalchemy.engine.Engine INSERT INTO books (title, price, memo) VALUES (?, ?, ?)
2024-02-09 14:28:48,922 INFO sqlalchemy.engine.Engine [generated in 0.00197s] ('やさしいPython', '2580', 'SBクリエイティブ')
2024-02-09 14:28:48,925 INFO sqlalchemy.engine.Engine COMMIT


**add\_all**()メソッドを使用すると一度に複数のBookインスタンスを渡してデータベースに登録出来ます。

In [13]:
session.add_all([
    Book(title= 'スッキリわかるJava入門', price='2680', memo = 'インプレス'),
    Book(title= '独習PHP', price='3520', memo = '翔泳社'),
])
session.commit()


2024-02-09 14:29:06,489 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-09 14:29:06,493 INFO sqlalchemy.engine.Engine INSERT INTO books (title, price, memo) VALUES (?, ?, ?) RETURNING id
2024-02-09 14:29:06,494 INFO sqlalchemy.engine.Engine [generated in 0.00022s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('スッキリわかるJava入門', '2680', 'インプレス')
2024-02-09 14:29:06,498 INFO sqlalchemy.engine.Engine INSERT INTO books (title, price, memo) VALUES (?, ?, ?) RETURNING id
2024-02-09 14:29:06,499 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('独習PHP', '3520', '翔泳社')
2024-02-09 14:29:06,503 INFO sqlalchemy.engine.Engine COMMIT


### R. オブジェクトの取得
オブジェクトの登録ができたら、取得を行います。オブジェクトの取得はsession.query()を実行してQueryオブジェクトを生成します。
QueryオブジェクトはSQLを実行するためのオブジェクトです。

In [5]:
query = session.query(Book)

NameError: name 'Book' is not defined

 **all**( )メソッドを使うと全てのオブジェクトを取得できます。

In [6]:
for book in query.all():
    print (book.title)
    

NameError: name 'query' is not defined

**get**( )メソッドを使うとプライマリーキーを用いてオブジェクトを取得できます。

In [16]:
book = query.get(1)
print( book.title)

2024-02-09 14:29:17,020 INFO sqlalchemy.engine.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.id = ?
2024-02-09 14:29:17,022 INFO sqlalchemy.engine.Engine [generated in 0.00256s] (1,)
やさしいPython


C:\Users\admin\AppData\Local\Temp\ipykernel_3772\845283479.py:1: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  book = query.get(1)


**filter**( )メソッドを使うと、条件を付けてオブジェクトを取得できます。

In [17]:
book = query.filter(Book.title == 'スッキリわかるJava入門').first()
print(book.title)

2024-02-09 14:30:52,591 INFO sqlalchemy.engine.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.title = ?
 LIMIT ? OFFSET ?
2024-02-09 14:30:52,591 INFO sqlalchemy.engine.Engine [generated in 0.00195s] ('スッキリわかるJava入門', 1, 0)
スッキリわかるJava入門


つぎのように**filter**( )メソッドをつなげて、複数の条件を指定することができます。

In [18]:
book = query.filter(Book.title == '独習PHP').filter(Book.id == 3).first()
print(book.title)

2024-02-09 14:30:55,496 INFO sqlalchemy.engine.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.title = ? AND books.id = ?
 LIMIT ? OFFSET ?
2024-02-09 14:30:55,498 INFO sqlalchemy.engine.Engine [generated in 0.00172s] ('独習PHP', 3, 1, 0)
独習PHP


**count**( )メソッドを使用すると、テーブル上のレコード数を取得できます。

In [19]:
count = query.count()
print(count)

2024-02-09 14:31:03,496 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books) AS anon_1
2024-02-09 14:31:03,498 INFO sqlalchemy.engine.Engine [generated in 0.00196s] ()
3


### U. オブジェクトの更新
オブジェクトの更新は**\.price=2500**のようにオブジェクトの値を変更して、**session\.commit**( )を呼び出します。

In [20]:
book = query.get(1)
book.price = 2500
session.commit()
book = query.get(1)
print(book.price)

2024-02-09 14:31:06,319 INFO sqlalchemy.engine.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.id = ?
2024-02-09 14:31:06,321 INFO sqlalchemy.engine.Engine [cached since 109.3s ago] (1,)
2024-02-09 14:31:06,326 INFO sqlalchemy.engine.Engine UPDATE books SET price=? WHERE books.id = ?
2024-02-09 14:31:06,327 INFO sqlalchemy.engine.Engine [generated in 0.00172s] (2500, 1)
2024-02-09 14:31:06,331 INFO sqlalchemy.engine.Engine COMMIT
2024-02-09 14:31:06,331 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-09 14:31:06,331 INFO sqlalchemy.engine.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.id = ?
2024-02-09 14:31:06,341 INFO sqlalchemy.engine.Engine [generated in 0.00335s] (1,)
2500


C:\Users\admin\AppData\Local\Temp\ipykernel_3772\747556896.py:1: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  book = query.get(1)
C:\Users\admin\AppData\Local\Temp\ipykernel_3772\747556896.py:4: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  book = query.get(1)


### D. オブジェクトの削除
オブジェクトの削除は**session\.delete**( )にBookインスタンスを渡してから、**session\.commit**( )を呼び出します。

In [21]:
book = query.get(1)
session.delete(book)
session.commit()
book = query.get(1)
print(book)

for book in query.all():
    print(book.title)

2024-02-09 14:31:10,099 INFO sqlalchemy.engine.Engine DELETE FROM books WHERE books.id = ?
2024-02-09 14:31:10,101 INFO sqlalchemy.engine.Engine [generated in 0.00188s] (1,)
2024-02-09 14:31:10,103 INFO sqlalchemy.engine.Engine COMMIT
2024-02-09 14:31:10,108 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-09 14:31:10,110 INFO sqlalchemy.engine.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.id = ?
2024-02-09 14:31:10,111 INFO sqlalchemy.engine.Engine [cached since 113.1s ago] (1,)
None
2024-02-09 14:31:10,116 INFO sqlalchemy.engine.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books
2024-02-09 14:31:10,117 INFO sqlalchemy.engine.Engine [cached since 117.3s ago] ()
スッキリわかるJava入門
独習PHP


C:\Users\admin\AppData\Local\Temp\ipykernel_3772\700329404.py:1: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  book = query.get(1)
C:\Users\admin\AppData\Local\Temp\ipykernel_3772\700329404.py:4: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  book = query.get(1)


In [22]:
book

<Book('独習PHP', '3520', '翔泳社')>